In [1]:
!nvidia-smi

Wed Aug 12 02:47:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

Cloning into 'dnn_from_scratch'...
remote: Enumerating objects: 214, done.
remote: Counting objects: 100% (214/214), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 1432 (delta 137), reused 148 (delta 83), pack-reused 1218
Receiving objects: 100% (1432/1432), 13.74 MiB | 23.14 MiB/s, done.
Resolving deltas: 100% (952/952), done.


In [3]:
!git clone https://github.com/ShivamShrirao/deep_Q_learning_from_scratch.git

Cloning into 'deep_Q_learning'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 111 (delta 61), reused 88 (delta 38), pack-reused 0
Receiving objects: 100% (111/111), 62.85 KiB | 1.14 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd deep_Q_learning_from_scratch

/content/deep_Q_learning


In [6]:
!git pull

Already up to date.


In [7]:
import gym
import time
import cv2
import numpy as np

from settings import *
from agent import *
from experience import *
from atari_wrappers import *

In [8]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-6)
D_exp = ReplayMemory(capacity=600000)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [9]:
env = gym.make('Pong-v0')
env = FrameStack(env, NFRAMES)      # preprocess and stack frames

In [10]:
len(D_exp.ndone)

0

In [ ]:
for i_episode in range(2000):
    observation = env.reset()
    ep_score = 0
    start = time.time()
    sv_obs = deque([], maxlen=NFRAMES)
    sv_act = deque([], maxlen=NFRAMES)
    sv_rew = deque([], maxlen=NFRAMES)
    sv_don = deque([], maxlen=NFRAMES)
    for t in range(10000):
#         env.render()
        action = agt.get_action(observation)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        sv_obs.append(observation)
        sv_act.append(action)
        sv_rew.append(reward)
        sv_don.append(done)

        D_exp.store_transition(sv_obs[0], agt.actions.index(sv_act[0]), sv_rew[0], next_observation, sv_don[0])
        observation = next_observation

        # D_exp.current_state.popleft()
        # D_exp.action_idx.popleft()
        # D_exp.reward.popleft()
        # D_exp.next_state.popleft()
        # D_exp.ndone.popleft()

        if len(D_exp.ndone) > BATCH_SIZE:
            agt.train(D_exp, BATCH_SIZE)
        
        if done:
            break
        print('\r', t, action, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon}, Time: {time.time()-start}")
    if not i_episode%8:
        agt.model.save_weights("/content/drive/My Drive/model.w8s")
env.close()

Episode 1 finished after 1433 timesteps, Score: -21.0, Epsilon: 0.989811999999707, Time: 29.253997087478638
Episode 2 finished after 1172 timesteps, Score: -21.0, Epsilon: 0.9886399999996733, Time: 23.76466965675354
Episode 3 finished after 1269 timesteps, Score: -21.0, Epsilon: 0.9873709999996368, Time: 25.96623682975769
Episode 4 finished after 1470 timesteps, Score: -18.0, Epsilon: 0.9859009999995946, Time: 30.172743797302246
Episode 5 finished after 1340 timesteps, Score: -21.0, Epsilon: 0.984560999999556, Time: 27.45044755935669
Episode 6 finished after 1385 timesteps, Score: -20.0, Epsilon: 0.9831759999995162, Time: 28.4764564037323
Episode 7 finished after 1431 timesteps, Score: -21.0, Epsilon: 0.9817449999994751, Time: 29.92097282409668
Episode 8 finished after 1367 timesteps, Score: -20.0, Epsilon: 0.9803779999994358, Time: 28.446946144104004
Episode 9 finished after 1327 timesteps, Score: -20.0, Epsilon: 0.9790509999993976, Time: 27.816803455352783
Episode 10 finished after 1

In [ ]:
env.close()